# Argentic RAG 
Dieser Prototyp untersucht die Fähigkeiten von LLM unterstützen Multiagent System zur Analyse von Mediationstranskriptionen. 

Als Framework wird crewai verwendet. 


In [8]:
# %pip install crewai
# %pip install load_dotenv
# %pip install os
# %pip install crewai[tools]
# %pip install agentops

Setup der Fachfragen an die Transkripte

In [9]:
Fall = "F5"


DomainFragen = ["Gib die Momente im Text mit Zeitstempel, bei der die Stimmung sehr stark wechselt?",
           "Welche Mediationsmethoden werden angewendet und an welchen Stellen?",
           "Nach welchen Äußerung des Mediators hat sich die Stimmung der Medianten stark verändert?",
           "In Welcher Session ist der Fokus in Richtung Lösung erkennbar?",
           "Wieviele Sitzung werden in dieser Mediation benötigt?", 
           "War die Mediation erfolgreich?", 
           "Wenn es zwei Mediatoren gibt, wie kooperieren sie?",
           "Wann werden die Phasen und Prinzipien der Mediation im Transkript erläutert?", 
           "Nenne alle Auffälligkeiten und die konkreten Stellen?", 
           "Wie steuert der Mediator die Mediation, gib Beispiele und die konkreten Stellen an?"]



Initialisierung der Dokumente

In [10]:
from crewai import Agent, Task, Crew, Process 
import os
from crewai_tools import DirectorySearchTool, PDFSearchTool, SerperDevTool
from langchain_openai import ChatOpenAI

os.environ["OPENAI_MODEL_NAME"]="gpt-4o"

doctool = DirectorySearchTool(directory=f'anon\\F5')

import agentops
agentops.init()


Inserting batches in chromadb:   0%|          | 0/21 [00:00<?, ?it/s]2024-05-27 12:10:43,736 - 24800 - sqlite.py-sqlite:284 - WARNING: Insert of existing embedding ID: default-app-id--56b6b176f719875a6a5b30ec24835aa8e1ced98ebefa4f9e8fd609ac53a9bd42
2024-05-27 12:10:43,738 - 24800 - sqlite.py-sqlite:284 - WARNING: Insert of existing embedding ID: default-app-id--681be5b89c214a6746cfcad4e6bac1712e8b405ff107056b92aa583fe0376cfa
2024-05-27 12:10:43,740 - 24800 - sqlite.py-sqlite:284 - WARNING: Insert of existing embedding ID: default-app-id--0b71e7cc759356c36b45020b9510cd6b2d17dd71c20e2fb37b6f2f99d390c718
2024-05-27 12:10:43,742 - 24800 - sqlite.py-sqlite:284 - WARNING: Insert of existing embedding ID: default-app-id--0a2ec08690fed4f1d450f88324f4c35b740efca1c09c73f3c725623f0bf59d07
2024-05-27 12:10:43,744 - 24800 - sqlite.py-sqlite:284 - WARNING: Insert of existing embedding ID: default-app-id--c0114a2ad7979721933841f34a5cda994216f889187821e8c9b03b241c59d21e
2024-05-27 12:10:43,747 - 24800

UUID('45eac718-5d05-46dd-9f7f-9ff823699054')

## Starten der Untersuchung


In [11]:
DomainExpert = Agent(
    role = 'Mediationsexperte',
    goal = 'Ziel ist die Feststellung der Arbeitsweise des Mediators in der konkreten Mediation',
    backstory = 'Du bist eine Mediationsexperte und kennst alle Praktiken der Mediation und kannst diese erkennen',
    verbose = True,
    allow_delegation= False,
    tools = [doctool]
)

TechnicalExpert = Agent(
    role = 'Technikexperte',
    goal = 'Ziel ist die NLP Analyse der Transkriptionen zur Bewertung der Arbeitsweise des Mediators',
    backstory = 'Du bist eine NLP Experte und kannst die Transkriptionen sprachlich analysieren.',
    verbose = True,
    allow_delegation= False,
    tools = [doctool] )

Writer = Agent(
    role = 'Schreiber',
    goal = 'Ziel ist ein genauer umfangreicher Bericht mit den der Anlysen der Experten zu verfassen.',
    backstory = 'Du bist eine sehr guter Autor und kannst komplexe Informationen verständlich formulieren. Weiterhin legst du viel Wert auf Evidenzen, und kannst die Textbeispiele klar belegen, sowie den Kontext mit jeweils 5 Zeilen wiedergeben',
    verbose = True,
    allow_delegation= False,
    tools = [doctool]
    )

Fragen = ""
for Frage in DomainFragen:
    Fragen+=f"{Frage}, "

research_task = Task(
    description=f'Untersuche die Arbeitsweise des Mediators in den Transkripten aus sprachlicher Sicht. Finde Besonderheiten und Stimmungsmomente',
    expected_output='NLP Analyse der Transkriptionen. Besonderheiten der Rederichtung, Redeanteile und Sentiments über den Zeitverlauf der einzelnen Sprecher an konkreten Beispielen und Zeilennummern', 
    agent=TechnicalExpert,
    tools=[doctool],
    output_file= f"Ergebnisse\\{Fall}\\{Fall}_aRAG_NLP.txt")

analysis_task  = Task( 
    description=f'Untersuche die Arbeitsweise des Mediators in den Transkripten aus Fallsicht und beantworte dabei die Fragen : {Fragen[:-2]}',
    expected_output='Angewandte Medationsmethoden und Interessante Wendepunkte im Transkript. Was wird wo verwendet, an konkreten Beispielen mit Sitzung (Textdatei) und dem Zeitstempel (Am Ende jeder Zeile)', 
    agent=DomainExpert,
    tools=[doctool],
    output_file= f"Ergebnisse\\{Fall}\\{Fall}_aRAG_Expert.txt")

writing_task = Task( 
    description= 'Schreibe einen sehr umfangreichen Bericht über die Arbeitsweise des Mediators im konketen Mediationstranskriptes in deutsch. Begründe die Analyse mit Beispielen und konkreten Zeilennummer oder Zeiten',
    expected_output=f'Anworenten auf die Fragen:  {Fragen[:-2]} , sowie angewandte Medationsmethoden und interessante Wendepunkte im Transkript. Was wird wo verwendet, an konkreten Beispielen mit Zeitstempel', 
    agent=Writer,
    tools=[doctool], 
    output_file= f"Ergebnisse\\{Fall}\\{Fall}_aRAG_Bericht.txt")

In [12]:
hier_report_crew = Crew(
  agents=[TechnicalExpert, DomainExpert, Writer],
  tasks=[research_task, analysis_task, writing_task],
  process=Process.hierarchical,
  manager_llm=ChatOpenAI(model="gpt-4o"),
  verbose=2,
  memory=False,
  cache=True,
)

2024-05-27 12:12:04,024 - 24800 - __init__.py-__init__:518 - WARNING: Overriding of current TracerProvider is not allowed
2024-05-27 12:12:04,646 - 24800 - __init__.py-__init__:518 - WARNING: Overriding of current TracerProvider is not allowed


In [1]:
result = hier_report_crew.kickoff()

print("######################")
print(result)
print("######################")

NameError: name 'hier_report_crew' is not defined

Starten der Untersuchung

In [ ]:
print(hier_report_crew.usage_metrics)

{'total_tokens': 219253, 'prompt_tokens': 203297, 'completion_tokens': 15956, 'successful_requests': 114}
